# COCO Dataset Generator
> forked from [here](https://www.kaggle.com/coldfir3/efficient-coco-dataset-generator)

In [ ]:
!pip install -Uqqq pycocotools

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from pathlib import Path
from pycocotools.coco import COCO
from PIL import Image

# Params

In [ ]:
FOLD = 0
dataDir=Path('../input/sartorius-cell-instance-segmentation/train')

# Meta Data

In [ ]:
df = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')
df.head()

# Data Split

In [ ]:
df = df.reset_index(drop=True)
df['fold'] = -1
skf = GroupKFold(n_splits=5)
for fold, (_, val_idx) in enumerate(skf.split(X=df, y=df['cell_type'], groups=df['id'])):
    df.loc[val_idx, 'fold'] = fold

# Helper

In [ ]:
## Based on: https://www.kaggle.com/eigrad/convert-rle-to-bounding-box-x0-y0-x1-y1
def rle2mask(rle, img_w, img_h):
    
    ## transforming the string into an array of shape (2, N)
    array = np.fromiter(rle.split(), dtype = np.uint)
    array = array.reshape((-1,2)).T
    array[0] = array[0] - 1
    
    ## decompressing the rle encoding (ie, turning [3, 1, 10, 2] into [3, 4, 10, 11, 12])
    # for faster mask construction
    starts, lenghts = array
    mask_decompressed = np.concatenate([np.arange(s, s + l, dtype = np.uint) for s, l in zip(starts, lenghts)])

    ## Building the binary mask
    msk_img = np.zeros(img_w * img_h, dtype = np.uint8)
    msk_img[mask_decompressed] = 1
    msk_img = msk_img.reshape((img_h, img_w))
    msk_img = np.asfortranarray(msk_img) ## This is important so pycocotools can handle this object
    
    return msk_img

## Minor Sanity Check

In [ ]:
rle = df.loc[0, 'annotation']
print(rle)
plt.imshow(rle2mask(rle, 704, 520));

# COCO Annotations

In [ ]:
from tqdm.notebook import tqdm
from pycocotools import mask as maskUtils
from joblib import Parallel, delayed
import json,itertools

def annotate(idx, row, cat_ids):
        mask = rle2mask(row['annotation'], row['width'], row['height']) # Binary mask
        c_rle = maskUtils.encode(mask) # Encoding it back to rle (coco format)
        c_rle['counts'] = c_rle['counts'].decode('utf-8') # converting from binary to utf-8
        area = maskUtils.area(c_rle).item() # calculating the area
        bbox = maskUtils.toBbox(c_rle).astype(int).tolist() # calculating the bboxes
        annotation = {
            'segmentation': c_rle,
            'bbox': bbox,
            'area': area,
            'image_id':row['id'], 
            'category_id':1, # cat_ids[row['cell_type']], 
            'iscrowd':0, 
            'id':idx
        }
        return annotation
    
def coco_structure(df, workers = 4):
    
    ## Building the header
    cat_ids = {"cell":1}    
    cats =[{'name':name, 'id':id} for name,id in cat_ids.items()]
    images = [{'id':id, 'width':row.width, 'height':row.height, 'file_name':f'{id}.png'}\
              for id,row in df.groupby('id').agg('first').iterrows()]
    
    ## Building the annotations
    annotations = Parallel(n_jobs=workers)(delayed(annotate)(idx, row, cat_ids) for idx, row in tqdm(df.iterrows(), total = len(df)))
        
    return {'categories':cats, 'images':images, 'annotations':annotations}

In [ ]:
train_df = df.query("fold!=@FOLD")
valid_df = df.query("fold==@FOLD")

In [ ]:
train_json = coco_structure(train_df)
valid_json = coco_structure(valid_df)

In [ ]:
display(train_json['annotations'][0])
display(valid_json['annotations'][0])

In [ ]:
with open('annotations_train.json', 'w', encoding='utf-8') as f:
    json.dump(train_json, f, ensure_ascii=True, indent=4)
with open('annotations_valid.json', 'w', encoding='utf-8') as f:
    json.dump(valid_json, f, ensure_ascii=True, indent=4)

# COCO Image

In [ ]:
!mkdir -p train2017
!mkdir -p valid2017

In [ ]:
import shutil
def run_copy(row):
    img_path = dataDir/f'{row.id}.png'
    if row.fold!=FOLD:
        shutil.copy(img_path, './train2017/')
    else:
        shutil.copy(img_path, './valid2017/')

In [ ]:
tmp_df = df.groupby('id').agg('first').reset_index()
_ = Parallel(n_jobs=-1,
         backend='threading')(delayed(run_copy)(row) for _, row in tqdm(tmp_df.iterrows(),
                                                                        total=len(tmp_df)))

# Sanity check

In [ ]:
# annFile = Path('./annotations_train.json')
# coco = COCO(annFile)
# imgIds = coco.getImgIds()

# imgs = coco.loadImgs(imgIds[-3:])
# _,axs = plt.subplots(len(imgs),2,figsize=(40,15 * len(imgs)))
# for img, ax in zip(imgs, axs):
#     I = Image.open(dataDir/img['file_name'])
#     annIds = coco.getAnnIds(imgIds=[img['id']])
#     anns = coco.loadAnns(annIds)
#     ax[0].imshow(I)
#     ax[1].imshow(I)
#     plt.sca(ax[1])
#     coco.showAnns(anns, draw_bbox=True)
# plt.tight_layout()
# plt.show()

In [ ]:
annFile = Path('./annotations_valid.json')
coco = COCO(annFile)
imgIds = coco.getImgIds()

imgs = coco.loadImgs(imgIds[-3:])
_,axs = plt.subplots(len(imgs),2,figsize=(40,15 * len(imgs)))
for img, ax in zip(imgs, axs):
    I = Image.open(dataDir/img['file_name'])
    annIds = coco.getAnnIds(imgIds=[img['id']])
    anns = coco.loadAnns(annIds)
    ax[0].imshow(I)
    ax[1].imshow(I)
    plt.sca(ax[1])
    coco.showAnns(anns, draw_bbox=True)
plt.tight_layout()
plt.show()

# Compress files

In [ ]:
shutil.make_archive('valid2017', 'zip', 'valid2017')
shutil.make_archive('train2017', 'zip', 'train2017')

In [ ]:
!rm -r train2017
!rm -r valid2017